# Slurp in Reference Headers

In [19]:
# imports
import os
from pkg_resources import resource_filename
import glob

from astropy.io import fits
from astropy.table import Table

from linetools import utils as ltu

In [21]:
out_path = os.path.join(resource_filename('pypeit', 'data'), 'spectrographs', 'keck_lris_red_mark4')

# Files

In [3]:
ref_path = '/scratch/REDUX/Keck/LRIS/new_LRISr/Reference_headers'

In [5]:
pixel_files = glob.glob(ref_path+'/pixels*')
pixel_files

['/scratch/REDUX/Keck/LRIS/new_LRISr/Reference_headers/pixelsU2_2_2.fits',
 '/scratch/REDUX/Keck/LRIS/new_LRISr/Reference_headers/pixelsL2U2L1U1_2_2.fits',
 '/scratch/REDUX/Keck/LRIS/new_LRISr/Reference_headers/pixelsU1_1_1.fits',
 '/scratch/REDUX/Keck/LRIS/new_LRISr/Reference_headers/pixelsL2U2_2_1.fits',
 '/scratch/REDUX/Keck/LRIS/new_LRISr/Reference_headers/pixelsL1U1_1_1.fits',
 '/scratch/REDUX/Keck/LRIS/new_LRISr/Reference_headers/pixelsL2U2_1_2.fits',
 '/scratch/REDUX/Keck/LRIS/new_LRISr/Reference_headers/pixelsU1_2_1.fits',
 '/scratch/REDUX/Keck/LRIS/new_LRISr/Reference_headers/pixelsL1_2_1.fits',
 '/scratch/REDUX/Keck/LRIS/new_LRISr/Reference_headers/pixelsU2U1_2_2.fits',
 '/scratch/REDUX/Keck/LRIS/new_LRISr/Reference_headers/pixelsL2L1_1_1.fits',
 '/scratch/REDUX/Keck/LRIS/new_LRISr/Reference_headers/pixelsL1U1_2_1.fits',
 '/scratch/REDUX/Keck/LRIS/new_LRISr/Reference_headers/pixelsU2U1_2_1.fits',
 '/scratch/REDUX/Keck/LRIS/new_LRISr/Reference_headers/pixelsU1_1_2.fits',
 '/sc

# Dict for names

In [16]:
ampmodes = Table.read('translate_ampmode.ascii', format='ascii')
ampmodes

AMPMODE,REF
str13,str8
"HSPLIT,VSPLIT",L2U2L1U1
"HSPLIT,VUP",L2U2
"HSPLIT,VDOWN",L1U1
"HLEFT,VSPLIT",L2L1
"HLEFT,VUP",L2
"HLEFT,VDOWN",L1
"HRIGHT,VSPLIT",U2U1
"HRIGHT,VUP",U2
"HRIGHT,VDOWN",U1


## dict me

In [18]:
ref_dict = {}
for row in ampmodes:
    ref_dict[row['AMPMODE']] = row['REF']
ref_dict

{'HSPLIT,VSPLIT': 'L2U2L1U1',
 'HSPLIT,VUP': 'L2U2',
 'HSPLIT,VDOWN': 'L1U1',
 'HLEFT,VSPLIT': 'L2L1',
 'HLEFT,VUP': 'L2',
 'HLEFT,VDOWN': 'L1',
 'HRIGHT,VSPLIT': 'U2U1',
 'HRIGHT,VUP': 'U2',
 'HRIGHT,VDOWN': 'U1'}

In [20]:
jdict = ltu.jsonify(ref_dict)

## Write

In [23]:
json_file = os.path.join(out_path, 'dict_for_ampmode.json')
ltu.savejson(json_file, jdict, overwrite=True, easy_to_read=True)

# Grab and save the Headers

In [25]:
for ifile in pixel_files:
    hdul = fits.open(ifile)
    head0 = hdul[0].header
    # New hdul
    new_hdul = fits.PrimaryHDU()
    new_hdul.header = head0
    # Outfile
    outfile = os.path.join(out_path, os.path.basename(ifile).replace('pixels', 'header'))
    new_hdul.writeto(outfile)

# Test

In [6]:
hdul = fits.open(pixel_files[0])

In [8]:
hdul[0].header[0:10]

SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                   16 / number of bits per data pixel                  
NAXIS   =                    2 / number of data axes                            
NAXIS1  =                 2117 / length of data axis 1                          
NAXIS2  =                 2124 / length of data axis 2                          
EXTEND  =                    T / FITS dataset may contain extensions            
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
PSCPIX1 =                    7 / [CCDpix] detector prescan pixels               
IMCPIX1 =                 4114 / [CCDpix] detector image pixels                 

In [9]:
img = fits.PrimaryHDU()

In [10]:
img.header = hdul[0].header

In [11]:
img.writeto('tmp.fits')